In [3]:
from typing import List

In [4]:
Vector=List[float]

In [5]:
def fizz_buzz_encode(x:int)->Vector:
    if x%15 == 0:
        return [0,0,0,1]
    elif x%5 == 0:
        return [0,0,1,0]
    elif x%3 == 0:
        return [0,1,0,0]
    else:
        return [1,0,0,0]
print(fizz_buzz_encode(2))
print(fizz_buzz_encode(6))
print(fizz_buzz_encode(10))
print(fizz_buzz_encode(30))


[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]


In [6]:
def binary(x: int)-> Vector:
    binary: List[float]=[]
    for _ in range(10):
        binary.append(x%2)
        x=x//2
    
    return binary
print(binary(3))

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
xs=[binary(x) for x in range(101,1024)]
ys=[fizz_buzz_encode(x) for x in range(101,1024)]

In [8]:
import random
import tqdm
NUM_HIDDEN=25

network =[
    [[random.random() for _ in range(10+1)] for _ in range(NUM_HIDDEN)],
    [[random.random() for _ in range(NUM_HIDDEN+1)] for _ in range(4)]
]

In [14]:
from dl_tools import feed_forward, squared_distance, sqerror_gradient,gradient_step

learning_rate=1

with tqdm.trange(500) as t:
    for epoch in t:
        epoch_loss=0.0
        for x,y in zip(xs,ys):
            predicted = feed_forward(network,x)[-1]
            epoch_loss+=squared_distance(predicted,y)
            gradients=sqerror_gradient(network,x,y)
            
            network = [[gradient_step(neuron,grad,-learning_rate) 
                        for neuron,grad in zip(layer,layer_grad)]
                       for layer,layer_grad in zip(network,gradients)]

        t.set_description(f"loss: {epoch_loss:.2f}")


  0%|          | 0/500 [00:00<?, ?it/s]

loss: 20.95: 100%|██████████| 500/500 [02:35<00:00,  3.21it/s] 


In [15]:
#Test
predicted = feed_forward(network,binary(905))[-1]

In [16]:
print(predicted)

[0.0024672287918810713, 0.0010586430017606864, 0.9998208029819575, 7.915153481791678e-10]


In [17]:
def argmax(xs: list) -> int:
    return max(range(len(xs)), key=lambda i: xs[i])

assert argmax([0.1,0.7,0.2])==1

In [19]:
#FizzBuzz
num_correct=0
for x in range(1,101):
    predicted=argmax(feed_forward(network,binary(x))[-1])
    actual=argmax(fizz_buzz_encode(x))
    labels=["fizz","buzz","fizzbuzz","other"]
    if predicted==actual:
        num_correct+=1
    else:
        print(x,labels[predicted],labels[actual])
print(num_correct,"/100 correct")

93 fizz buzz
99 /100 correct
